In [10]:
import os
import glob
import numpy as np
import cv2

In [11]:
# img_dir = '../images/'
img_dir = '../4_images/'

In [12]:
all_image_names = ['image_' + str(i) for i in range(118)]
ext_dict = {
    'og': 'original/',
    'all': 'all_edges/',
    '0': '0/',
    '30': '30/',
    '60': '60/',
    '90': '90/',
    '120': '120/',
    '150': '150/'
}
orientation_angle_dict = {
    '0': 0,  # 0 degrees
    '30': np.pi / 6,  # 30 degrees
    '60': np.pi / 3,  # 60 degrees
    '90': np.pi / 2,  # 90 degrees
    '120': 2 * np.pi / 3,  # 120 degrees
    '150': 5 * np.pi / 6  # 150 degrees
}
orientations = [0, 30, 60, 90, 120, 150]

In [13]:
for ext in ext_dict.values():
    if not os.path.exists(img_dir + ext):
        os.makedirs(img_dir + ext)

In [14]:
def get_paths(dir, image_type='png'):
    """
    get the file name/path of all the files within a folder.
        e.g. glob.glob("/home/adam/*/*.txt").
    Use glob.escape to escape strings that are not meant to be patterns
        glob.glob(glob.escape(directory_name) + "/*.txt")
    :param directory_name: (str) the root directory name that contains all the images we want
    :param image: (str) either "jpg" or "png"
    :return: a list of queried files and directories
    """
    end = "/*." + image_type
    file_paths = glob.glob(glob.escape(dir) + end)
    return file_paths

In [15]:
originals = get_paths(img_dir + ext_dict['og'])

In [16]:
# noise reduction threshold
noise_threshold = 0.1
gaussianblur_kernel_size = 13
# intensity thresholds 
t1 = 50
t2 = 100
# angle error threshold
angle_error_threshold = np.pi / 12 # 15 degrees

In [17]:
def generate_edge_by_orientation(all_edges, gradient_angle, orientation):
    # Retrieve the orientation angle and the corresponding save ext
    orientation_angle = orientation_angle_dict[orientation]
    # subtract 90 degrees from the angle
    orientation_angle -= np.pi / 2
    # Compute orientation mask
    edge_mask = np.logical_and(gradient_angle > orientation_angle - angle_error_threshold,
                               gradient_angle < orientation_angle + angle_error_threshold)
    # Apply mask and save result
    result = all_edges.copy()
    result[~edge_mask] = 0
    cv2.imwrite(os.path.join(img_dir, ext_dict[orientation] + file_name), result)

In [18]:
# For each original image...
for f in originals: 
    if '57' in f or '61' in f or '81' in f or '44' in f:
        # Read image in greyscale
        img = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
        assert img is not None, "file could not be read, check with os.path.exists()"
        # Compute all edges and save result
        blur = cv2.GaussianBlur(src=img, ksize=(gaussianblur_kernel_size, gaussianblur_kernel_size), sigmaX=0)
        all_edges = cv2.Canny(image=blur, threshold1=t1, threshold2=t2, apertureSize=3)
        # file_name = f[19:]
        file_name = f[21:]
        cv2.imwrite(os.path.join(img_dir, ext_dict['all'] + file_name), all_edges)
        # Compute gradient angle using Sobel with kernel size 3
        sobelx = cv2.Sobel(blur, cv2.CV_64F, 1, 0, ksize=3)
        sobely = cv2.Sobel(blur, cv2.CV_64F, 0, 1, ksize=3)
        gradient_angle = np.arctan2(sobely, sobelx)
        # For each orientation...
        for o in orientations:
            orientation = str(o)
            # Generate edges and save result
            generate_edge_by_orientation(all_edges, gradient_angle, orientation)